In [109]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

Data Pre-Processing

1. Load the CSV into a DataFrame

In [110]:
df = pd.read_csv("imdb_top_movies.csv")

In [111]:
df.head()

,Title,Year,Rating,Genre,Director(s),Box Office Revenue,Lead Actors
0,1. The Shawshank Redemption,1994,9.3 (3M),"Epic, Period Drama, Prison Drama, Drama","The Ink Spots, Jack Lawrence, Stephen King, Ti...","Gross worldwide$29,332,133","Bob Gunton, Morgan Freeman, Tim Robbins"
1,2. The Godfather,1972,9.2 (2.1M),"Epic, Gangster, Tragedy, Crime, Drama","Mario Puzo, Marlon Brando, Peter Clemenza, Al ...","Gross worldwide$250,342,198","Al Pacino, Marlon Brando, James Caan"
2,3. The Dark Knight,2008,9.0 (3M),"Action Epic, Epic, Superhero, Tragedy, Action,...","Aaron Eckhart, Michael Caine, The Joker, Jonat...","Gross worldwide$1,009,057,329","Aaron Eckhart, Christian Bale, Heath Ledger"
3,4. The Godfather Part II,1974,9.0 (1.4M),"Epic, Gangster, Tragedy, Crime, Drama","Mario Puzo, Francesco Pennino, Al Pacino, Robe...","Gross worldwide$47,964,222","Al Pacino, Robert De Niro, Robert Duvall"
4,5. 12 Angry Men,1957,9.0 (917K),"Legal Drama, Psychological Drama, Crime, Drama","Henry Fonda, Jack Warden, Lee J. Cobb, Juror #...","Gross worldwide$2,945","Henry Fonda, Martin Balsam, Lee J. Cobb"


2. Data Cleaning and Type Conversion

In [112]:
type(df)

pandas.core.frame.DataFrame

In [113]:
df.shape

(250, 7)

In [114]:
#Null Value check

df.isna().sum()

Title                 0
Year                  0
Rating                0
Genre                 0
Director(s)           0
Box Office Revenue    0
Lead Actors           0
dtype: int64

In [115]:
#Dropping Duplicates
df= df.drop_duplicates()
df

,Title,Year,Rating,Genre,Director(s),Box Office Revenue,Lead Actors
0,1. The Shawshank Redemption,1994,9.3 (3M),"Epic, Period Drama, Prison Drama, Drama","The Ink Spots, Jack Lawrence, Stephen King, Ti...","Gross worldwide$29,332,133","Bob Gunton, Morgan Freeman, Tim Robbins"
1,2. The Godfather,1972,9.2 (2.1M),"Epic, Gangster, Tragedy, Crime, Drama","Mario Puzo, Marlon Brando, Peter Clemenza, Al ...","Gross worldwide$250,342,198","Al Pacino, Marlon Brando, James Caan"
2,3. The Dark Knight,2008,9.0 (3M),"Action Epic, Epic, Superhero, Tragedy, Action,...","Aaron Eckhart, Michael Caine, The Joker, Jonat...","Gross worldwide$1,009,057,329","Aaron Eckhart, Christian Bale, Heath Ledger"
3,4. The Godfather Part II,1974,9.0 (1.4M),"Epic, Gangster, Tragedy, Crime, Drama","Mario Puzo, Francesco Pennino, Al Pacino, Robe...","Gross worldwide$47,964,222","Al Pacino, Robert De Niro, Robert Duvall"
4,5. 12 Angry Men,1957,9.0 (917K),"Legal Drama, Psychological Drama, Crime, Drama","Henry Fonda, Jack Warden, Lee J. Cobb, Juror #...","Gross worldwide$2,945","Henry Fonda, Martin Balsam, Lee J. Cobb"
...,...,...,...,...,...,...,...
245,246. A Silent Voice: The Movie,2016,8.1 (117K),"Anime, Coming-of-Age, Psychological Drama, Shō...","Saori Hayami, Naoko Yamada, Lexi Marman, Shoko...","Gross worldwide$30,819,442","Miyu Irino, Saori Hayami, Aoi Yûki"
246,247. The Help,2011,8.1 (510K),"Period Drama, Drama","Hilly Holbrook, Emma Stone, Jerry Leiber, Tate...","Gross worldwide$221,802,186","Emma Stone, Octavia Spencer, Viola Davis"
247,248. Amores Perros,2000,8.0 (261K),"Tragedy, Drama, Thriller","Emilio Echevarría, Guillermo Arriaga, Alejandr...","Gross worldwide$20,908,467","Goya Toledo, Gael García Bernal, Emilio Echeva..."
248,249. Rebecca,1940,8.1 (153K),"Dark Romance, Psychological Drama, Psychologic...","Laurence Olivier, Mrs. Danvers, The Second Mrs...","Gross worldwide$113,328","Laurence Olivier, George Sanders, Joan Fontaine"


In [116]:
df.dtypes

Title                 object
Year                   int64
Rating                object
Genre                 object
Director(s)           object
Box Office Revenue    object
Lead Actors           object
dtype: object

In [117]:
# 1. Split "Title" into "ID" and "Movie Title

# Add new columns "ID" and "Movie Title" to DataFrame
df["ID"] = id_title["ID"]
df["Movie Title"] = id_title["MovieTitle"].fillna(df["Title"]).str.strip()

# Remove the old "Title" column
df.drop(columns=["Title"], inplace=True)
df.head()

,Year,Rating,Genre,Director(s),Box Office Revenue,Lead Actors,ID,Movie Title
0,1994,9.3 (3M),"Epic, Period Drama, Prison Drama, Drama","The Ink Spots, Jack Lawrence, Stephen King, Ti...","Gross worldwide$29,332,133","Bob Gunton, Morgan Freeman, Tim Robbins",1,The Shawshank Redemption
1,1972,9.2 (2.1M),"Epic, Gangster, Tragedy, Crime, Drama","Mario Puzo, Marlon Brando, Peter Clemenza, Al ...","Gross worldwide$250,342,198","Al Pacino, Marlon Brando, James Caan",2,The Godfather
2,2008,9.0 (3M),"Action Epic, Epic, Superhero, Tragedy, Action,...","Aaron Eckhart, Michael Caine, The Joker, Jonat...","Gross worldwide$1,009,057,329","Aaron Eckhart, Christian Bale, Heath Ledger",3,The Dark Knight
3,1974,9.0 (1.4M),"Epic, Gangster, Tragedy, Crime, Drama","Mario Puzo, Francesco Pennino, Al Pacino, Robe...","Gross worldwide$47,964,222","Al Pacino, Robert De Niro, Robert Duvall",4,The Godfather Part II
4,1957,9.0 (917K),"Legal Drama, Psychological Drama, Crime, Drama","Henry Fonda, Jack Warden, Lee J. Cobb, Juror #...","Gross worldwide$2,945","Henry Fonda, Martin Balsam, Lee J. Cobb",5,12 Angry Men


In [118]:
# 1. Split "Rating" into separate columns for rating and viewers
extracted = df["Rating"].str.extract(r"^(.*?)\s*\((.*?)\)$")

# If the rating string has parentheses, group(1) is the rating part, group(2) is the viewers part.
df["Rating (raw)"] = extracted[0].fillna(df["Rating"])  # Fall back to original if no parentheses
df["Viewers"] = extracted[1]                            # This will be NaN if there's no match

# Convert "Rating (raw)" to numeric, coercing invalid entries to NaN
df["Rating (raw)"] = pd.to_numeric(df["Rating (raw)"], errors="coerce")

# Remove the old "Rating" column
df.drop(columns=["Rating"], inplace=True)

# Parse the "Viewers" column (e.g., "3.5M" -> 3,500,000; "500K" -> 500,000)
def parse_viewers(value):
    if pd.isna(value):
        return np.nan
    val_str = str(value).strip().upper()  # e.g., "3.5M" -> "3.5M"
    
    # Match a pattern like "3.5M" or "500K"
    match = re.match(r"^(\d+(\.\d+)?)([MK])$", val_str)
    if match:
        number_part = float(match.group(1))  # "3.5" or "500"
        suffix = match.group(3)             # "M" or "K"
        
        if suffix == "M":
            return number_part * 1_000_000
        elif suffix == "K":
            return number_part * 1_000
    
    return np.nan  # If it doesn't match our pattern, treat as unknown

df["Viewers"] = df["Viewers"].apply(parse_viewers)

# Inspect the cleaned columns
print(df[["Rating (raw)", "Viewers"]].head())

   Rating (raw)    Viewers
0           9.3  3000000.0
1           9.2  2100000.0
2           9.0  3000000.0
3           9.0  1400000.0
4           9.0   917000.0


In [119]:
# 3. Clean the "Box Office Revenue" column

df["Box Office Revenue"] = df["Box Office Revenue"].str.replace("Gross worldwide", "", regex=False)
df["Box Office Revenue"] = df["Box Office Revenue"].str.replace("$", "", regex=False)
df["Box Office Revenue"] = df["Box Office Revenue"].str.replace(",", "", regex=False)
df["Box Office Revenue"] = pd.to_numeric(df["Box Office Revenue"], errors="coerce")
df.head()

,Year,Genre,Director(s),Box Office Revenue,Lead Actors,ID,Movie Title,Rating (raw),Viewers
0,1994,"Epic, Period Drama, Prison Drama, Drama","The Ink Spots, Jack Lawrence, Stephen King, Ti...",2.933213e+07,"Bob Gunton, Morgan Freeman, Tim Robbins",1,The Shawshank Redemption,9.3,3000000.0
1,1972,"Epic, Gangster, Tragedy, Crime, Drama","Mario Puzo, Marlon Brando, Peter Clemenza, Al ...",2.503422e+08,"Al Pacino, Marlon Brando, James Caan",2,The Godfather,9.2,2100000.0
2,2008,"Action Epic, Epic, Superhero, Tragedy, Action,...","Aaron Eckhart, Michael Caine, The Joker, Jonat...",1.009057e+09,"Aaron Eckhart, Christian Bale, Heath Ledger",3,The Dark Knight,9.0,3000000.0
3,1974,"Epic, Gangster, Tragedy, Crime, Drama","Mario Puzo, Francesco Pennino, Al Pacino, Robe...",4.796422e+07,"Al Pacino, Robert De Niro, Robert Duvall",4,The Godfather Part II,9.0,1400000.0
4,1957,"Legal Drama, Psychological Drama, Crime, Drama","Henry Fonda, Jack Warden, Lee J. Cobb, Juror #...",2.945000e+03,"Henry Fonda, Martin Balsam, Lee J. Cobb",5,12 Angry Men,9.0,917000.0


In [120]:
# Remove the "ID" column
df.drop(columns=["ID"], inplace=True)
df.head()

,Year,Genre,Director(s),Box Office Revenue,Lead Actors,Movie Title,Rating (raw),Viewers
0,1994,"Epic, Period Drama, Prison Drama, Drama","The Ink Spots, Jack Lawrence, Stephen King, Ti...",2.933213e+07,"Bob Gunton, Morgan Freeman, Tim Robbins",The Shawshank Redemption,9.3,3000000.0
1,1972,"Epic, Gangster, Tragedy, Crime, Drama","Mario Puzo, Marlon Brando, Peter Clemenza, Al ...",2.503422e+08,"Al Pacino, Marlon Brando, James Caan",The Godfather,9.2,2100000.0
2,2008,"Action Epic, Epic, Superhero, Tragedy, Action,...","Aaron Eckhart, Michael Caine, The Joker, Jonat...",1.009057e+09,"Aaron Eckhart, Christian Bale, Heath Ledger",The Dark Knight,9.0,3000000.0
3,1974,"Epic, Gangster, Tragedy, Crime, Drama","Mario Puzo, Francesco Pennino, Al Pacino, Robe...",4.796422e+07,"Al Pacino, Robert De Niro, Robert Duvall",The Godfather Part II,9.0,1400000.0
4,1957,"Legal Drama, Psychological Drama, Crime, Drama","Henry Fonda, Jack Warden, Lee J. Cobb, Juror #...",2.945000e+03,"Henry Fonda, Martin Balsam, Lee J. Cobb",12 Angry Men,9.0,917000.0


In [121]:
df.dtypes

Year                    int64
Genre                  object
Director(s)            object
Box Office Revenue    float64
Lead Actors            object
Movie Title            object
Rating (raw)          float64
Viewers               float64
dtype: object